In [1]:
# All imports go here
import pandas as pd
from sqlalchemy import create_engine, text,  CursorResult
import psycopg
from time import time

In [2]:
# endpoint = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-01.csv.gz"
endpoint = "hw1/data/yellow_tripdata_2019-01.csv.gz"

In [3]:
df = pd.read_csv(endpoint, compression='gzip', nrows=100)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

In [4]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.60,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.00,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.00,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.00,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,2019-01-01 00:02:06,2019-01-01 00:10:19,2,2.57,1,N,45,97,1,9.5,0.5,0.5,2.16,0.0,0.3,12.96,NaN
96,2,2019-01-01 00:24:57,2019-01-01 00:35:42,2,2.44,1,N,144,256,2,10.5,0.5,0.5,0.00,0.0,0.3,11.80,NaN
97,2,2019-01-01 00:46:09,2019-01-01 01:06:46,1,4.18,1,N,79,263,1,16.0,0.5,0.5,3.46,0.0,0.3,20.76,NaN
98,1,2019-01-01 00:16:07,2019-01-01 00:19:41,1,1.00,1,N,140,233,1,5.0,0.5,0.5,1.00,0.0,0.3,7.30,NaN


In [5]:
user="postgres"
password="postgres"
db_name = "ny_taxi"
hostname="localhost"
DATABASE_URL = f"postgresql+psycopg://{user}:{password}@{hostname}:5433/{db_name}"

# Create an engine instance
engine = create_engine(DATABASE_URL)

In [6]:
'''
# Example usage: connect and execute a query
with engine.connect() as connection:
    result:list = connection.execute(text("SELECT datname FROM pg_database;")).all()
    total_rows = len(result)
    print("Total rows: [{total_rows}]")
    for index, row in enumerate(result, start=1):
        print(f"({index} of {total_rows}), [{row}]")
'''

'\n# Example usage: connect and execute a query\nwith engine.connect() as connection:\n    result:list = connection.execute(text("SELECT datname FROM pg_database;")).all()\n    total_rows = len(result)\n    print("Total rows: [{total_rows}]")\n    for index, row in enumerate(result, start=1):\n        print(f"({index} of {total_rows}), [{row}]")\n'

In [7]:
# Create the table
# Get the schema
schema = print(pd.io.sql.get_schema(df, name = "yellow_taxi_data", con = engine))
print(schema)


CREATE TABLE yellow_taxi_data (
	"VendorID" BIGINT, 
	tpep_pickup_datetime TIMESTAMP WITHOUT TIME ZONE, 
	tpep_dropoff_datetime TIMESTAMP WITHOUT TIME ZONE, 
	passenger_count BIGINT, 
	trip_distance FLOAT(53), 
	"RatecodeID" BIGINT, 
	store_and_fwd_flag TEXT, 
	"PULocationID" BIGINT, 
	"DOLocationID" BIGINT, 
	payment_type BIGINT, 
	fare_amount FLOAT(53), 
	extra FLOAT(53), 
	mta_tax FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	improvement_surcharge FLOAT(53), 
	total_amount FLOAT(53), 
	congestion_surcharge FLOAT(53)
)


None


In [12]:
# Write the results to db
df_iter = pd.read_csv(endpoint, iterator=True, chunksize=100000)
# df_iter = pd.read_csv("hw1/data/small.csv", iterator=True, chunksize=100000)

start_time = time()
while (df := next(df_iter, None)) is not None:
    t_start = time()
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.to_sql(name='yellow_taxi_data', con=engine, if_exists='append')
    t_end = time()
    print('inserted another chunk, took %.3f second' % (t_end - t_start))

end_time = time()
print(f"Final insertion took {end_time - start_time} seconds")

inserted another chunk, took 2.424 second
inserted another chunk, took 2.275 second
inserted another chunk, took 2.246 second
inserted another chunk, took 2.277 second
inserted another chunk, took 2.440 second
inserted another chunk, took 2.320 second
inserted another chunk, took 2.443 second
inserted another chunk, took 2.259 second
inserted another chunk, took 2.258 second
inserted another chunk, took 2.272 second
inserted another chunk, took 2.269 second
inserted another chunk, took 2.237 second
inserted another chunk, took 2.300 second
inserted another chunk, took 2.298 second
inserted another chunk, took 2.354 second
inserted another chunk, took 2.588 second
inserted another chunk, took 2.401 second
inserted another chunk, took 2.230 second
inserted another chunk, took 2.315 second
inserted another chunk, took 2.236 second
inserted another chunk, took 2.319 second
inserted another chunk, took 2.313 second
inserted another chunk, took 2.391 second
inserted another chunk, took 2.347

In [9]:
'''
# Get all the data
with engine.connect() as connection:
    result:list = connection.execute(text("SELECT * FROM yellow_taxi_data;")).all()
    total_rows = len(result)
    print(f"Total rows: [{total_rows}]")
    for index, row in enumerate(result, start=1):
        print(f"({index} of {total_rows}), [{row}]")
'''

'\n# Get all the data\nwith engine.connect() as connection:\n    result:list = connection.execute(text("SELECT * FROM yellow_taxi_data;")).all()\n    total_rows = len(result)\n    print(f"Total rows: [{total_rows}]")\n    for index, row in enumerate(result, start=1):\n        print(f"({index} of {total_rows}), [{row}]")\n'

In [11]:
# Get row count
with engine.connect() as connection:
    result:list = connection.execute(text("SELECT count(1) FROM yellow_taxi_data;")).all()
    total_rows = len(result)
    print(f"Total rows: [{total_rows}]")
    for index, row in enumerate(result, start=1):
        print(f"({index} of {total_rows}), [{row}]")

Total rows: [1]
(1 of 1), [(7667792,)]
